## PASSAGE EN CLASSE BINAIRE

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_palette("viridis")

In [2]:
accidents=pd.read_csv('accidents.csv', index_col='Num_Acc')
accidents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 764600 entries, 201800000001 to 202200055302
Data columns (total 28 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   an             764600 non-null  int64  
 1   mois           764600 non-null  int64  
 2   lum            764600 non-null  int64  
 3   agg            764600 non-null  int64  
 4   int            764600 non-null  int64  
 5   atm            764600 non-null  float64
 6   col            764600 non-null  float64
 7   dep            764600 non-null  object 
 8   heure          764600 non-null  int64  
 9   jour_semaine   764600 non-null  int64  
 10  catr           764600 non-null  float64
 11  circ           764600 non-null  float64
 12  surf           764600 non-null  float64
 13  infra          764600 non-null  float64
 14  situ           764600 non-null  float64
 15  senc           764600 non-null  float64
 16  obs            764600 non-null  float64
 17  obsm           76

C:\Users\lgfen\AppData\Local\Temp\ipykernel_8576\1784307817.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents=pd.read_csv('accidents.csv', index_col='Num_Acc')


In [3]:
accidents['grav'].value_counts()

grav
1.0    344549
4.0    290527
3.0    111186
2.0     18338
Name: count, dtype: int64

In [4]:
accidents['grav']=accidents['grav'].replace(1.0,0)

In [5]:
accidents['grav']=accidents['grav'].replace(4.0,0)
accidents['grav']=accidents['grav'].replace(2.0,1)
accidents['grav']=accidents['grav'].replace(3.0,1)

In [6]:
accidents['grav'].value_counts()

grav
0.0    635076
1.0    129524
Name: count, dtype: int64

In [7]:
# SEPARATION JEU DE DONNES
from sklearn.model_selection import train_test_split
y=accidents['grav']
X=accidents.drop('grav', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [8]:
# ENCODAGE
# Variables à encoder avec One-Hot Encoding
var_onehot = ['lum', 'agg', 'col', 'jour_semaine', 'catr', 'circ','situ',
                'obsm', 'catu', 'sexe', 'trajet', 'secu_combined']

# Variables à encoder avec Frequency Encoding
var_freq = ['dep', 'manv', 'obs', 'int', 'atm','surf', 'infra', 'choc', 'catv_modifié', 'place']
                                                                                                                                                                                                                                                                                                                                                                                                              
# Initialiser l'encodeur One-Hot
from sklearn.preprocessing import OneHotEncoder
enc_onehot = OneHotEncoder(sparse_output=False, drop=None) 

# Appliquer l'encodage et convertir en DataFrame
## X_train
X_train_enc = enc_onehot.fit_transform(X_train[var_onehot])
X_train_enc_df = pd.DataFrame(X_train_enc, columns=enc_onehot.get_feature_names_out(var_onehot), index=X_train.index)

# Concaténer avec le DataFrame original
X_train_enc_df = pd.concat([X_train.drop(columns=var_onehot), X_train_enc_df], axis=1)

#----------------------------------------------------------------------------------------------------------------------------
## X_test
X_test_enc=enc_onehot.transform(X_test[var_onehot])
X_test_enc_df = pd.DataFrame(X_test_enc, columns=enc_onehot.get_feature_names_out(var_onehot), index=X_test.index)

# Concaténer avec le DataFrame original
X_test_enc_df = pd.concat([X_test.drop(columns=var_onehot), X_test_enc_df], axis=1)

In [ ]:
# Frequency encoding
##X_train
for var in var_freq:
    # Calculer la fréquence de chaque catégorie
    freq_encoding = X_train_enc_df[var].value_counts() / len(X_train_enc_df)
    # Remplacer les catégories par leur fréquence
    X_train_enc_df[var] = X_train_enc_df[var].map(freq_encoding)

#--------------------------------------------------------------------------
##X_test
for var in var_freq:
    # Calculer la fréquence de chaque catégorie
    freq_encoding = X_test_enc_df[var].value_counts() / len(X_test_enc_df)
    # Remplacer les catégories par leur fréquence
    X_test_enc_df[var] = X_test_enc_df[var].map(freq_encoding)

In [ ]:
# STANDARDISATION
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train_enc_df)
X_test_scaled=scaler.transform(X_test_enc_df)

## Oversampling

In [11]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler()
X_train_over, y_train_over = ros.fit_resample(X_train_scaled, y_train)
print('Resampled dataset shape %s' % Counter(y_train_over))

Resampled dataset shape Counter({0.0: 507821, 1.0: 507821})


## MODELISATION

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from imblearn.metrics import classification_report_imbalanced

## Mapping

In [13]:
# Mapping des classes pour affichage lisible
labels_map = {0: "0 - Léger", 1: "1 - Grave"}

## 1 - Régression logistique

In [14]:
# Définition du modèle
logreg_over=LogisticRegression(solver='liblinear', max_iter=1000)

# Définition des hyperparamètres à tester
param_logreg = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'class_weight' : ['balanced', {0:2, 1:1}, {0:3, 1:1}]}

# GridSearchCV avec validation croisée
grid_search_over_logreg = GridSearchCV(logreg_over, param_logreg, cv=3, scoring='recall', n_jobs=-1)
grid_search_over_logreg.fit(X_train_over, y_train_over)

GridSearchCV(cv=3,
             estimator=LogisticRegression(max_iter=1000, solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10],
                         'class_weight': ['balanced', {0: 2, 1: 1},
                                          {0: 3, 1: 1}],
                         'penalty': ['l1', 'l2']},
             scoring='recall')

In [ ]:
# Affichage des meilleurs paramètres et du score associé
print("Meilleurs paramètres :", grid_search_over_logreg.best_params_, "\n")
print("Meilleur score :", grid_search_over_logreg.best_score_, "\n")

# Affichage du meilleur modèle et de ses prédictions
best_logreg = grid_search_over_logreg.best_estimator_
y_pred_over_logreg = best_logreg.predict(X_test_scaled)

y_pred_series = pd.Series(y_pred_over_logreg, index=y_test.index)
y_test_named = y_test.replace(labels_map)
y_pred_named = y_pred_series.replace(labels_map)

# Matrice de confusion
print("Matrice de confusion", "\n")
print(pd.crosstab(y_test_named, y_pred_named, rownames=['classe réelle'], colnames=['classe prédite']), "\n")

# Rapport de classification
print("Rapport de classification", "\n")
print(classification_report(y_test, y_pred_over_logreg, target_names=["0 - Léger", "1 - Grave"]), "\n")

Meilleurs paramètres : {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l1'} 

Meilleur score : 0.7715454856107374 

Matrice de confusion 

classe prédite  0 - Léger  1 - Grave
classe réelle                       
0 - Léger           90902      36353
1 - Grave            5802      19863 

Rapport de classification 

              precision    recall  f1-score   support

   0 - Léger       0.94      0.71      0.81    127255
   1 - Grave       0.35      0.77      0.49     25665

    accuracy                           0.72    152920
   macro avg       0.65      0.74      0.65    152920
weighted avg       0.84      0.72      0.76    152920
 



## 2 - Random forest

In [ ]:
# Définition du modèle
randomforest_over=RandomForestClassifier(random_state=42)

# Définition des hyperparamètres à tester
param_rand = {
    'n_estimators': [50, 100],
     'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'class_weight' : ['balanced', {0:2, 1:1}]}

# RandomizedSearchCV avec validation croisée
random_search_over_rf = RandomizedSearchCV(randomforest_over, param_rand, cv=3, scoring='recall', n_jobs=-1, n_iter=5, random_state=42)
random_search_over_rf.fit(X_train_over, y_train_over)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'class_weight': ['balanced',
                                                         {0: 2, 1: 1}],
                                        'max_depth': [5, 10, 20, None],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100]},
                   random_state=42, scoring='recall')

In [ ]:
# Affichage des meilleurs paramètres et du score associé
print("Meilleurs paramètres :", random_search_over_rf.best_params_, "\n")
print("Meilleur score :", random_search_over_rf.best_score_, "\n")

# Affichage du meilleur modèle et de ses prédictions
best_rf = random_search_over_rf.best_estimator_
y_pred_over_rf = best_rf.predict(X_test_scaled)

y_pred_series = pd.Series(y_pred_over_rf, index=y_test.index)
y_test_named = y_test.replace(labels_map)
y_pred_named = y_pred_series.replace(labels_map)

# Matrice de confusion
print("Matrice de confusion", "\n")
print(pd.crosstab(y_test_named, y_pred_named, rownames=['classe réelle'], colnames=['classe prédite']), "\n")

# Rapport de classification 
print("Rapport de classification", "\n")
print(classification_report(y_test, y_pred_over_rf, target_names=["0 - Léger", "1 - Grave"]), "\n")

Meilleurs paramètres : {'n_estimators': 100, 'min_samples_split': 2, 'max_depth': None, 'class_weight': {0: 2, 1: 1}} 

Meilleur score : 0.994714683316961 

Matrice de confusion 

classe prédite  0 - Léger  1 - Grave
classe réelle                       
0 - Léger          118518       8737
1 - Grave           12631      13034 

Rapport de classification 

              precision    recall  f1-score   support

   0 - Léger       0.90      0.93      0.92    127255
   1 - Grave       0.60      0.51      0.55     25665

    accuracy                           0.86    152920
   macro avg       0.75      0.72      0.73    152920
weighted avg       0.85      0.86      0.86    152920
 



## 3 - KNN

In [18]:
# Définition du modèle
knn_over = KNeighborsClassifier()

# Définition des hyperparamètres à tester
param_grid = {
    'n_neighbors': [3, 5 ,7, 9],  
    'metric': ['minkowski', 'manhattan'],  
    'weights': ['uniform', 'distance']  }

# RandomizedSearchCV avec validation croisée
random_search_over_knn = RandomizedSearchCV(knn_over, param_grid, cv=3, scoring='recall', n_jobs=-1, n_iter=5, random_state=42)
random_search_over_knn.fit(X_train_over, y_train_over)

RandomizedSearchCV(cv=3, estimator=KNeighborsClassifier(), n_iter=5, n_jobs=-1,
                   param_distributions={'metric': ['minkowski', 'manhattan'],
                                        'n_neighbors': [3, 5, 7, 9],
                                        'weights': ['uniform', 'distance']},
                   random_state=42, scoring='recall')

In [ ]:
# Affichage des meilleurs paramètres et du score associé
print("Meilleurs paramètres :", random_search_over_knn.best_params_, "\n")
print("Meilleur score :", random_search_over_knn.best_score_, "\n")

# Affichage du meilleur modèle et de ses prédictions
best_knn = random_search_over_knn.best_estimator_
y_pred_over_knn = best_knn.predict(X_test_scaled)

y_pred_series = pd.Series(y_pred_over_knn, index=y_test.index)
y_test_named = y_test.replace(labels_map)
y_pred_named = y_pred_series.replace(labels_map)

# Matrice de confusion
print("Matrice de confusion", "\n")
print(pd.crosstab(y_test_named, y_pred_named, rownames=['classe réelle'], colnames=['classe prédite']), "\n")

# Rapport de classification 
print("Rapport de classification", "\n")
print(classification_report(y_test, y_pred_over_knn, target_names=["0 - Léger", "1 - Grave"]), "\n")

Meilleurs paramètres : {'weights': 'distance', 'n_neighbors': 7, 'metric': 'manhattan'} 

Meilleur score : 0.9965499721529328 

Matrice de confusion 

classe prédite  0 - Léger  1 - Grave
classe réelle                       
0 - Léger           97404      29851
1 - Grave            7123      18542 

Rapport de classification 

              precision    recall  f1-score   support

   0 - Léger       0.93      0.77      0.84    127255
   1 - Grave       0.38      0.72      0.50     25665

    accuracy                           0.76    152920
   macro avg       0.66      0.74      0.67    152920
weighted avg       0.84      0.76      0.78    152920
 



## 4 - XGBoost

In [20]:
# Définition du modèle
xgb_clf = XGBClassifier(objective='binary:logistic', eval_metric='mlogloss',scale_pos_weight=1)

# Définition des hyperparamètres à tester
param_grid = {
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
    'n_estimators': [50, 100],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# GridSearchCV avec validation croisée
grid_search_over_xgb = GridSearchCV(xgb_clf, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid_search_over_xgb.fit(X_train_over, y_train_over)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constrain...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.8, 1.0],
                         'learning_rate': [0.01, 0.1], 'max_depth': [3, 5],
                         'n_estimators': [50, 100], 'subsample': [0.8, 1.0]},
             scoring='recall')

In [ ]:
# Affichage des meilleurs paramètres et du score associé
print("Meilleurs paramètres :", grid_search_over_xgb.best_params_, "\n")
print("Meilleur score :", grid_search_over_xgb.best_score_, "\n")

# Affichage du meilleur modèle et de ses prédictions
best_xgb = grid_search_over_xgb.best_estimator_
y_pred_over_xgb = best_xgb.predict(X_test_scaled)

y_pred_series = pd.Series(y_pred_over_xgb, index=y_test.index)
y_test_named = y_test.replace(labels_map)
y_pred_named = y_pred_series.replace(labels_map)

# Matrice de confusion
print("Matrice de confusion", "\n")
print(pd.crosstab(y_test_named, y_pred_named, rownames=['classe réelle'], colnames=['classe prédite']), "\n")

# Rapport de classification
print("Rapport de classification", "\n")
print(classification_report(y_test, y_pred_over_xgb, target_names=["0 - Léger", "1 - Grave"]), "\n")

Meilleurs paramètres : {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0} 

Meilleur score : 0.8005576726568243 

Matrice de confusion 

classe prédite  0 - Léger  1 - Grave
classe réelle                       
0 - Léger           53388      73867
1 - Grave            2335      23330 

Rapport de classification 

              precision    recall  f1-score   support

   0 - Léger       0.96      0.42      0.58    127255
   1 - Grave       0.24      0.91      0.38     25665

    accuracy                           0.50    152920
   macro avg       0.60      0.66      0.48    152920
weighted avg       0.84      0.50      0.55    152920
 

